In [1]:
# import library
!pip install --no-cache-dir -q \
    torch torchvision torchaudio \
      -f https://download.pytorch.org/whl/cu124/torch_stable.html \
    transformers peft accelerate bitsandbytes \
    sentence-transformers scikit-learn rank-bm25 \
    networkx tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 285.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 273.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 255.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 323.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 270.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 257.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 298.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 275.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 244.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 273.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 239.1 MB/s eta 0:00:00


In [2]:
!pip install --no-cache-dir -q gdown

# Download the entire DATA structure directly from Drive
!gdown --folder "https://drive.google.com/drive/folders/1sdgi9g5uuXARsLzveDwoeoi4xKf0eg4V?usp=sharing" -O /content/drive-download

from pathlib import Path
import shutil, json, os, zipfile, pickle, random

# Everything is now located in /content/drive-download
drive_root = "/content/drive-download"
work_root  = "/content/KGP-CuriousLLM"
data_root  = f"{work_root}/DATA"

os.makedirs(data_root + "/KGs/graph_hotpot", exist_ok=True)
os.makedirs(data_root + "/HotpotQA", exist_ok=True)

# Knowledge graph
src_graph = f"{drive_root}/KGs/graph_hotpot/graph.gpickle"
dst_graph = f"{data_root}/KGs/graph_hotpot/graph.gpickle"
shutil.copy2(src_graph, dst_graph)

# Indexed questions file
src_qs = f"{drive_root}/HotpotQA/indexed_test_docs.json"
dst_qs = f"{data_root}/HotpotQA/indexed_test_docs.json"
shutil.copy2(src_qs, dst_qs)

print("Files copied.")

Retrieving folder contents
Retrieving folder 1PLoZwIa6Xt7mg0nDpnS-frtcmMsykFFv 2WikiMQA
Processing file 1J21Q-IHl0xHFVrBIwI3mql4L--gaVybD test_docs.json
Processing file 1NNXN0zefq27FWkSrHSKEfszoyn30iY-k indexed_test_docs.json
Processing file 17BU22PtNwidv44k5ynvq3DJUepyIQsOv indexed_2wiki_test_questions.json
Retrieving folder 1SZWdSWhomEtaMFsG5k5wOZ1aFb7DDZWA HotpotQA
Processing file 18rIwClQgkMgdWFHnF5wzaQjVZkmBeZ6Q val_with_neg_v0.json
Processing file 1a4dVGJ24cdQ88Ikz-vt3wFlxOREJlgLE test_docs.json
Processing file 1dgGbtiM4MlSAjudq7FuvSBGbCn8HpxIz indexed_test_docs.json
Processing file 1Ir8KHBVWghMMzQPQlgae1jdrEdXxom9g indexed_hotpot_test_questions.json
Retrieving folder 1bSwzX4KViZOev2HjPc1Tcy07G3P2Jqj4 KGs
Retrieving folder 14eKJ6f4hKDDGfOGu-CvgJY5XIHDKS8w1 graph_2wiki
Processing file 1hDaDFX0ay5-xRlW_z83tat4XzfwDTUQx config.yml
Processing file 1jF7iD7szvEvDsFXdW-URJD7GekmezQpf graph.gpickle
Retrieving folder 1q27c-8cKUG7n3V6xuqYRVxPriBvJMOrY graph_hotpot
Processing file 13CKqF-p0

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch, gc

base_model_name  = "Qwen/Qwen3-8B"  # Base model
lora_repo_name   = "mohammad-shirkhani/Qwen3_finetune_follow_up_question"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, lora_repo_name)
model.eval();  gc.collect();  torch.cuda.empty_cache()

print("Qwen + LoRA loaded")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/349M [00:00<?, ?B/s]

Qwen + LoRA loaded


In [4]:
qwen_args = {
    "seed"      : 2048,
    "device"    : "cuda",
    "n_hop"     : 2,
    "n_neighbors": 3,
    "generate": {   # Generate parameters
        "temperature": 0.6,
        "top_p"      : 0.85,
        "max_tokens" : 50,
    },
    "paths": {
        "graph"      : dst_graph,
        "questions"  : dst_qs,
        "save_dir"   : f"{data_root}/KG/evidence/hotpot_evidence_100/qwen_agent",
    }
}
Path(qwen_args["paths"]["save_dir"]).mkdir(parents=True, exist_ok=True)
print(json.dumps(qwen_args, indent=2))

{
  "seed": 2048,
  "device": "cuda",
  "n_hop": 2,
  "n_neighbors": 3,
  "generate": {
    "temperature": 0.6,
    "top_p": 0.85,
    "max_tokens": 50
  },
  "paths": {
    "graph": "/content/KGP-CuriousLLM/DATA/KGs/graph_hotpot/graph.gpickle",
    "questions": "/content/KGP-CuriousLLM/DATA/HotpotQA/indexed_test_docs.json",
    "save_dir": "/content/KGP-CuriousLLM/DATA/KG/evidence/hotpot_evidence_100/qwen_agent"
  }
}


In [5]:
# Qwen Agent (generates follow-up q)
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
import numpy as np
from collections import deque
from tqdm import tqdm
import random, json, torch

# seed retriever: TF-IDF
class TFIDFSeedRetriever:
    def __init__(self, graph, topk=4):
        self.topk = topk
        passages = ["TITLE: " + d["title"] + ". " + d["passage"]
                    for _, d in sorted(graph.nodes(data=True))]
        self.vectorizer = TfidfVectorizer()
        self.mat = self.vectorizer.fit_transform(passages)

    def retrieve(self, query):
        qv = self.vectorizer.transform([query])
        sims = cosine_similarity(qv, self.mat).flatten()
        return sims.argsort()[-self.topk:][::-1]

# Qwen Agent
class QwenAgent:
    def __init__(self, model, tokenizer, device, gen_cfg,
                 emb_model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1"):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.gen_cfg = gen_cfg
        self.emb = SentenceTransformer(emb_model_name, device=device)

    @torch.no_grad()
    def ask(self, prompt: str) -> str:
        template = (
            "You are a critical thinker and like to ask follow-up questions.\n"
            "Question context:\n" + prompt.strip() + "\n\nFollow-up question:"
        )
        inputs = self.tokenizer(
            template,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(self.device)

        out = self.model.generate(
            **inputs,
            do_sample=True,
            temperature=self.gen_cfg["temperature"],
            top_p=self.gen_cfg["top_p"],
            max_new_tokens=self.gen_cfg["max_tokens"],
            repetition_penalty=1.1,
            pad_token_id=self.tokenizer.pad_token_id
        )
        text = self.tokenizer.decode(out[0], skip_special_tokens=True)
        if "follow-up question:" in text.lower():
            text = text.split("question:", 1)[-1]
        return text.strip()

    def top_neighbors(self, G, path, main_question, k):
        prompt = (
            "Question: " + main_question +
            "\nEvidence: " + " ".join(G.nodes[n]["passage"] for n in path)
        )
        follow_q = self.ask(prompt)

        if follow_q.strip().lower() in {"na", "n/a", "none"}:
            return []

        # similarity on GPU
        q_emb = self.emb.encode(follow_q, convert_to_tensor=True)            # (768,)
        neigh_ids = list(G.neighbors(path[-1]))
        passages = [
            "TITLE: " + G.nodes[n]["title"] + ". " + G.nodes[n]["passage"]
            for n in neigh_ids
        ]
        neigh_emb = self.emb.encode(passages, convert_to_tensor=True)        # (N,768)

        scores = util.dot_score(q_emb, neigh_emb).cpu().numpy().flatten()
        best = scores.argsort()[-k:][::-1]                                   # Descending
        return [neigh_ids[i] for i in best]


# Searching the graph and collecting evidence
def collect_evidence(G, questions, seed_ret, agent, args):
    random.seed(args["seed"])
    evidence_records = []

     # Split indices to Bridge / Comparison according to the list in repo
    bridge_ids = set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20,
                      22, 23, 24, 25, 26, 28, 29, 35, 36, 37, 38, 39, 47, 54, 55, 57,
                      58, 60, 64, 65, 66, 68, 70, 72, 73, 79, 80, 81, 82, 84, 88])
    comparison_ids = set([13, 18, 21, 27, 31, 32, 33, 34, 42, 43, 44, 46, 50, 59, 62,
                          75, 76, 96, 117, 118, 124, 128, 162, 174, 178, 191, 197, 227,
                          237, 244, 253, 261, 262, 272, 273, 283, 285, 296, 299, 307,
                          317, 318, 319, 338, 342, 348, 353, 354, 356, 357])

    for idx, qrec in tqdm(questions, total=len(questions)):
        qtxt = qrec["question"]

        # Seed stage
        seed_nodes = seed_ret.retrieve(qtxt)
        queue = deque([([s], 0) for s in seed_nodes])
        visited = set(seed_nodes)

        while queue:
            path, depth = queue.popleft()
            if depth >= args["n_hop"]:
                continue
            next_nodes = agent.top_neighbors(G, path, qtxt, args["n_neighbors"])
            for n in next_nodes:
                if n not in visited:
                    visited.add(n)
                    queue.append((path + [n], depth + 1))

        ev_strings = [
            f"Title: {G.nodes[n]['title']}. Evidence: {G.nodes[n]['passage']}"
            for n in visited
        ]
        qtype = "bridge" if idx in bridge_ids else "comparison" if idx in comparison_ids else "unknown"

        evidence_records.append({
            "type"     : qtype,
            "question" : qtxt,
            "answer"   : qrec["answer"],
            "evidence" : ev_strings,
            "supports" : qrec["supports"]
        })

    return evidence_records

In [6]:
# Load the graph and question dataset
import random
import json
import pickle
from pathlib import Path

# paths from your qwen_args dict
graph_path    = qwen_args["paths"]["graph"]
question_path = qwen_args["paths"]["questions"]

# 1) Load the graph via pickle.load
with open(graph_path, "rb") as f:
    G = pickle.load(f)

print(f"Graph loaded  |  Nodes: {G.number_of_nodes():,}  Edges: {G.number_of_edges():,}")

# 2) Load the questions JSON
with open(question_path, "r") as f:
    all_qs = json.load(f)

bridge_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20,
                      22, 23, 24, 25, 26, 28, 29, 35, 36, 37, 38, 39, 47, 54, 55, 57,
                      58, 60, 64, 65, 66, 68, 70, 72, 73, 79, 80, 81, 82, 84, 88]
comparison_ids = [13, 18, 21, 27, 31, 32, 33, 34, 42, 43, 44, 46, 50, 59, 62,
                          75, 76, 96, 117, 118, 124, 128, 162, 174, 178, 191, 197, 227,
                          237, 244, 253, 261, 262, 272, 273, 283, 285, 296, 299, 307,
                          317, 318, 319, 338, 342, 348, 353, 354, 356, 357]
random.seed(qwen_args["seed"])
chosen = bridge_ids[:50] + comparison_ids[:50]
random.shuffle(chosen)

questions = [(i, all_qs[i]) for i in chosen]
print(f"{len(questions)} questions shuffled and ready.")

/tmp/ipython-input-6-389348078.py:13: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  G = pickle.load(f)


Graph loaded  |  Nodes: 263,847  Edges: 3,834,332
100 questions shuffled and ready.


In [7]:
# Create seed retriever and agent-
seed_retriever = TFIDFSeedRetriever(G, topk=4)
agent = QwenAgent(
    model, tokenizer,
    device=qwen_args["device"],
    gen_cfg=qwen_args["generate"],
    emb_model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
)

# Collect evidence
records = collect_evidence(G, questions, seed_retriever, agent, qwen_args)

# Save
out_file = Path(qwen_args["paths"]["save_dir"]) / "evidence.json"
with open(out_file, "w") as f:
    json.dump(records, f, indent=4)

print(f"✔ Evidence saved to {out_file}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 100/100 [1:46:08<00:00, 63.69s/it]

✔ Evidence saved to /content/KGP-CuriousLLM/DATA/KG/evidence/hotpot_evidence_100/qwen_agent/evidence.json


In [8]:
# 1) Paths for the 2Wiki dataset
wiki_graph_src   = f"{drive_root}/KGs/graph_2wiki/graph.gpickle"
wiki_graph_dst   = f"{data_root}/KGs/graph_2wiki/graph.gpickle"

wiki_qs_src      = f"{drive_root}/2WikiMQA/indexed_test_docs.json"
wiki_qs_dst      = f"{data_root}/2WikiMQA/indexed_test_docs.json"

Path(f"{data_root}/KGs/graph_2wiki").mkdir(parents=True, exist_ok=True)
if not Path(wiki_graph_dst).exists():
    shutil.copy2(wiki_graph_src, wiki_graph_dst)

Path(f"{data_root}/2WikiMQA").mkdir(parents=True, exist_ok=True)
if not Path(wiki_qs_dst).exists():
    shutil.copy2(wiki_qs_src, wiki_qs_dst)

print("2Wiki graph & questions ready.")

2Wiki graph & questions ready.


In [9]:
# 2) Special hyperparameters for 2Wiki
wiki_args = {
    "seed"      : 2048,
    "device"    : "cuda",
    "n_hop"     : 2,
    "n_neighbors": 3,
    "generate": {
        "temperature": 0.6,
        "top_p"      : 0.85,
        "max_tokens" : 50,
    },
    "paths": {
        "graph"     : wiki_graph_dst,
        "questions" : wiki_qs_dst,
        "save_dir"  : f"{data_root}/KG/evidence/wiki_evidence_100/qwen_agent",
    }
}
Path(wiki_args["paths"]["save_dir"]).mkdir(parents=True, exist_ok=True)
print(json.dumps(wiki_args, indent=2))

{
  "seed": 2048,
  "device": "cuda",
  "n_hop": 2,
  "n_neighbors": 3,
  "generate": {
    "temperature": 0.6,
    "top_p": 0.85,
    "max_tokens": 50
  },
  "paths": {
    "graph": "/content/KGP-CuriousLLM/DATA/KGs/graph_2wiki/graph.gpickle",
    "questions": "/content/KGP-CuriousLLM/DATA/2WikiMQA/indexed_test_docs.json",
    "save_dir": "/content/KGP-CuriousLLM/DATA/KG/evidence/wiki_evidence_100/qwen_agent"
  }
}


In [10]:
# 3) List of Bridge / Comparison indexes for 2Wiki
bridge_ids_2wiki = [
    0, 1, 2, 11, 12, 13, 14, 18, 22, 28, 29, 31, 32, 33, 35, 38, 39, 41, 42, 43,
    45, 46, 51, 53, 54, 55, 56, 57, 58, 64, 65, 67, 68, 69, 73, 74, 77, 79, 80,
    82, 83, 86, 87, 91, 92, 97, 98, 100, 104, 106
]
comparison_ids_2wiki = [
    3, 4, 5, 6, 7, 8, 10, 15, 16, 17, 19, 24, 25, 26, 30, 34, 37, 47, 48, 50,
    59, 60, 61, 62, 63, 70, 71, 72, 76, 78, 84, 85, 89, 90, 93, 94, 95, 96, 99,
    107, 114, 115, 116, 117, 119, 128, 139, 141, 145, 150
]

# 100 questions
random.seed(wiki_args["seed"])
chosen_wiki = bridge_ids_2wiki[:50] + comparison_ids_2wiki[:50]
random.shuffle(chosen_wiki)


# 4) Load graph and questions
with open(wiki_args["paths"]["graph"], "rb") as f:
    G_wiki = pickle.load(f)
print(f"Wiki-Graph | Nodes: {G_wiki.number_of_nodes():,}  Edges: {G_wiki.number_of_edges():,}")

with open(wiki_args["paths"]["questions"], "r") as f:
    all_qs_wiki = json.load(f)

questions_wiki = [(i, all_qs_wiki[i]) for i in chosen_wiki]
print(f"{len(questions_wiki)} questions (50 bridge + 50 comparison)")

/tmp/ipython-input-10-132403187.py:21: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  G_wiki = pickle.load(f)


Wiki-Graph | Nodes: 112,037  Edges: 1,610,969
100 questions (50 bridge + 50 comparison)


In [11]:
# 5) Create Retriever and Agent
seed_ret_wiki = TFIDFSeedRetriever(G_wiki, topk=4)
agent_wiki    = QwenAgent(
    model, tokenizer,
    device=wiki_args["device"],
    gen_cfg=wiki_args["generate"],
    emb_model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
)

# 6) Traverse the graph and collect evidence
records_wiki = collect_evidence(
    G_wiki,
    questions_wiki,
    seed_ret_wiki,
    agent_wiki,
    wiki_args
)

# 7) Save output
wiki_out = Path(wiki_args["paths"]["save_dir"]) / "evidence.json"
with open(wiki_out, "w") as f:
    json.dump(records_wiki, f, indent=4)

print(f"Evidence saved to {wiki_out}")

100%|██████████| 100/100 [1:35:15<00:00, 57.15s/it]

Evidence saved to /content/KGP-CuriousLLM/DATA/KG/evidence/wiki_evidence_100/qwen_agent/evidence.json


In [12]:
# 1) Mount your Google Drive (in Colab)
from google.colab import drive
drive.mount('/content/drive')

# 2) Define source & target paths
from pathlib import Path
import shutil

hotpot_src = Path('/content/KGP-CuriousLLM/DATA/KG/evidence/hotpot_evidence_100/qwen_agent/evidence.json')
wiki_src   = Path('/content/KGP-CuriousLLM/DATA/KG/evidence/wiki_evidence_100/qwen_agent/evidence.json')
target_dir = Path('/content/drive/MyDrive/KGP_evidence')

# 3) Create the target directory if it doesn't exist
target_dir.mkdir(parents=True, exist_ok=True)

# 4) Copy & rename the evidence files into Drive
shutil.copy2(hotpot_src, target_dir / 'hotpotqa_evidence_qwen.json')
shutil.copy2(wiki_src,   target_dir / '2wiki_evidence_qwen.json')

print(f"All evidence files have been moved to {target_dir}")

Mounted at /content/drive
All evidence files have been moved to /content/drive/MyDrive/KGP_evidence
